In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as functional
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as py
import time

##%matplotlib inline

input_size = 28
num_classes = 10
num_epochs = 3
batch_size = 64

##制作训练集和测试集
train_dataset = datasets.MNIST(root='./DATA', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(root='DATA', train=False, transform=transforms.ToTensor())

##制作batch数据集
train_load = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_load = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(1, 16, 5, 1, 2), nn.ReLU(), nn.MaxPool2d(2))

        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, 5, 1, 2), nn.ReLU(), nn.MaxPool2d(2))

        ##self.conv3 = nn.Sequential(nn.Conv2d(32, 64, 5, 1, 2), nn.ReLU(), nn.MaxPool2d(2))

        self.out = nn.Linear(32 * 7 * 7, 10)
        ##self.out = nn.Linear(64 * 3 * 3, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        ##x = self.conv3(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output


def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1]
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)


net = CNN()

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=0.001)

time_begin = time.time()

for epoch in range(num_epochs):
    train_rights = []

    for batch_idx, (data, target) in enumerate(train_load):
        net.train()
        output = net(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output, target)
        train_rights.append(right)

        if batch_idx % 100 == 0:
            net.eval()
            val_rights = []

            for (data, target) in test_load:
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))

            print('epoch: {} [{}/{} ({:.0f}%)]\t loss: {:.6f}\t train accuracy: {:.2f}%\t right rate: {:.2f}%'.format(
                epoch, batch_idx * batch_size, len(train_load.dataset),
                       100. * batch_idx / len(train_load),
                loss.data,
                       100. * train_r[0].numpy() / train_r[1],
                       100. * val_r[0].numpy() / val_r[1]))
time_end = time.time()

print(time_end - time_begin,'s')

epoch: 0 [0/60000 (0%)]	 loss: 2.320711	 train accuracy: 6.25%	 right rate: 16.38%
epoch: 0 [6400/60000 (11%)]	 loss: 0.260307	 train accuracy: 73.53%	 right rate: 92.47%
epoch: 0 [12800/60000 (21%)]	 loss: 0.259206	 train accuracy: 83.61%	 right rate: 95.38%
epoch: 0 [19200/60000 (32%)]	 loss: 0.089080	 train accuracy: 87.55%	 right rate: 96.79%
epoch: 0 [25600/60000 (43%)]	 loss: 0.283640	 train accuracy: 89.73%	 right rate: 96.52%
epoch: 0 [32000/60000 (53%)]	 loss: 0.062103	 train accuracy: 91.12%	 right rate: 97.39%
epoch: 0 [38400/60000 (64%)]	 loss: 0.018309	 train accuracy: 92.11%	 right rate: 97.53%
epoch: 0 [44800/60000 (75%)]	 loss: 0.127803	 train accuracy: 92.84%	 right rate: 97.89%
epoch: 0 [51200/60000 (85%)]	 loss: 0.048107	 train accuracy: 93.44%	 right rate: 98.08%
epoch: 0 [57600/60000 (96%)]	 loss: 0.061304	 train accuracy: 93.91%	 right rate: 97.93%
epoch: 1 [0/60000 (0%)]	 loss: 0.044280	 train accuracy: 98.44%	 right rate: 98.06%
epoch: 1 [6400/60000 (11%)]	 loss

In [ ]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

batch_size = 64

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307, ), (0.3081, ))
])#把[]中的操作整成一个pipline,均值和标准差

train_dataset = datasets.MNIST(root='./dataset/mnist/',
                                train=True,
                                download=True,
                                transform=transform)
train_loader = DataLoader(train_dataset,
                          shuffle=True,
                          batch_size=batch_size)
test_dataset = datasets.MNIST(root='./dataset/mnist/',
                              train=False,
                              download=True,
                              transform=transform)
test_loader = DataLoader(test_dataset,
                         shuffle=False,
                         batch_size=batch_size)

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5)
        self.pooling = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(320, 10)
    def forward(self, x):
    # Flatten data from (n, 1, 28, 28) to (n, 784)
        batch_size = x.size(0)
        x = F.relu(self.pooling(self.conv1(x)))
        x = F.relu(self.pooling(self.conv2(x)))
        x = x.view(batch_size, -1)  # flatten
        x = self.fc(x)
        return x
model = Net()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        # forward + backward + update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 300))
            running_loss = 0.0

def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, target = data
            inputs, target = inputs.to(device), target.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, dim=1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
        print('Accuracy on test set: %d %%' % (100 * correct / total))#%%输出就是%，转义
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()



  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./dataset/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./dataset/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./dataset/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./dataset/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./dataset/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./dataset/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./dataset/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./dataset/mnist/MNIST/raw

